In [1]:
# !pip install fastapi uvicorn mysql-connector-python python-dotenv
# !pip install watchgod

In [ ]:
import requests

response = requests.get("http://localhost:3001/api/sales?year=2023")
if response.status_code == 200:
    print("Monthly Sales:", response.json())
else:
    print("Error:", response.status_code, response.text)


In [2]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
import mysql.connector
from dotenv import load_dotenv
import os

load_dotenv()

app = FastAPI()

# Enable CORS (adjust origins as needed)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # In production, use the specific domain
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Database connection
def get_connection():
    return mysql.connector.connect(
        host=os.getenv("DB_HOST"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        database=os.getenv("DB_NAME"),
    )

@app.get("/api/sales")
def get_sales_by_year(year: int):
    try:
        conn = get_connection()
        cursor = conn.cursor(dictionary=True)

        query = """
            SELECT 
              MONTH(date) AS month,
              SUM(total_earn) AS total_earn
            FROM sales
            WHERE YEAR(date) = %s
            GROUP BY MONTH(date)
            ORDER BY MONTH(date);
        """
        cursor.execute(query, (year,))
        results = cursor.fetchall()

        # Fill missing months with 0
        monthly_totals = [0.0] * 12
        for row in results:
            monthly_totals[row["month"] - 1] = float(row["total_earn"])

        return monthly_totals

    except mysql.connector.Error as err:
        print("MySQL Error:", err)
        raise HTTPException(status_code=500, detail="Database error")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()


In [ ]:
from fastapi import FastAPI, HTTPException
from typing import List
import mysql.connector
from mysql.connector import errorcode
from pydantic import BaseModel
import os

app = FastAPI()

# Load your RDS connection info from environment variables or config
RDS_HOST = os.getenv("RDS_HOST", "your-rds-host")
RDS_USER = os.getenv("RDS_USER", "your-username")
RDS_PASSWORD = os.getenv("RDS_PASSWORD", "your-password")
RDS_DB = os.getenv("RDS_DB", "your-database")

class ProductStock(BaseModel):
    ProductID: str
    Balance: int

def get_db_connection():
    try:
        conn = mysql.connector.connect(
            host=RDS_HOST,
            user=RDS_USER,
            password=RDS_PASSWORD,
            database=RDS_DB
        )
        return conn
    except mysql.connector.Error as err:
        print(f"Error connecting to DB: {err}")
        return None

@app.get("/inventory/{store_id}", response_model=List[ProductStock])
def get_inventory(store_id: str):
    conn = get_db_connection()
    if not conn:
        raise HTTPException(status_code=500, detail="Database connection error")

    cursor = conn.cursor(dictionary=True)

    # Query to get latest date per store, and get all products' balances for that date
    # Adjust table/column names to your actual schema
    query = """
        SELECT ProductID, Balance FROM inventory_data
        WHERE StoreID = %s
          AND Date = (
            SELECT MAX(Date) FROM inventory_data WHERE StoreID = %s
          )
        ORDER BY ProductID
    """

    cursor.execute(query, (store_id, store_id))
    results = cursor.fetchall()
    cursor.close()
    conn.close()

    # Return list of dict {ProductID, Balance}
    return [ProductStock(**row) for row in results]


FileNotFoundError: [Errno 2] No such file or directory: 'inventory_data.csv'

In [9]:
import requests

BASE_URL = "http://127.0.0.1:3333"

def test_sales(year: int):
    try:
        response = requests.get(f"{BASE_URL}/api/sales", params={"year": year})
        response.raise_for_status()
        print(f"\n✅ Sales Data for {year}:")
        print(response.json())
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching sales data: {e}")

def test_inventory():
    try:
        response = requests.get(f"{BASE_URL}/api/inventory")
        response.raise_for_status()
        print("\n✅ Inventory Data:")
        for record in response.json():
            print(record)
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching inventory data: {e}")
        
        
def test_tables():
    try:
        response = requests.get(f"{BASE_URL}/api/tables")
        response.raise_for_status()
        print("\n✅ Tables:")
        for record in response.json():
            print(record)
    except requests.exceptions.RequestException as e:
        print(f"❌ Error fetching inventory data: {e}")


In [10]:
table_infos = test_tables()


✅ Tables:
['product']
['qbi_file_20250517_13_04_46_0']
['qbi_file_20250517_13_05_05_0']
['qbi_file_20250517_21_37_08_0']
['qbi_file_20250517_21_45_35_0']
['sales']
['user']


In [25]:
test_sales(2024)


✅ Sales Data for 2024:
[50343.2, 205084.19999999995, 327273.0000000001, 424259.6000000001, 205477.40000000002, 389137.60000000015, 326802.79999999993, 375692.39999999985, 397557.3999999999, 491170.3999999999, 385433.59999999986, 339866.4000000001]


In [26]:
test_inventory()


✅ Inventory Data:
{'Date': '2025-05-16T00:00:00', 'ProductID': 'P025', 'StoreID': 'ST002', 'QtyIn': 0.0, 'QtyOut': 9.0, 'Balance': 216.0}
{'Date': '2025-05-16T00:00:00', 'ProductID': 'P002', 'StoreID': 'ST001', 'QtyIn': 0.0, 'QtyOut': 2.0, 'Balance': 345.0}
{'Date': '2025-05-16T00:00:00', 'ProductID': 'P018', 'StoreID': 'ST001', 'QtyIn': 0.0, 'QtyOut': 10.0, 'Balance': 140.0}
{'Date': '2025-05-16T00:00:00', 'ProductID': 'P029', 'StoreID': 'ST002', 'QtyIn': 0.0, 'QtyOut': 8.0, 'Balance': 62.0}
{'Date': '2025-05-16T00:00:00', 'ProductID': 'P024', 'StoreID': 'ST001', 'QtyIn': 0.0, 'QtyOut': 5.0, 'Balance': 34.0}
{'Date': '2025-05-16T00:00:00', 'ProductID': 'P007', 'StoreID': 'ST001', 'QtyIn': 0.0, 'QtyOut': 0.0, 'Balance': 0.0}
{'Date': '2025-05-16T00:00:00', 'ProductID': 'P012', 'StoreID': 'ST001', 'QtyIn': 0.0, 'QtyOut': 1.0, 'Balance': 21.0}
{'Date': '2025-05-16T00:00:00', 'ProductID': 'P048', 'StoreID': 'ST002', 'QtyIn': 0.0, 'QtyOut': 1.0, 'Balance': 22.0}
{'Date': '2025-05-16T00:00

In [2]:
import os
from http import HTTPStatus
from dashscope import Application
import dashscope
from datetime import datetime

dashscope.base_http_api_url = 'https://dashscope-intl.aliyuncs.com/api/v1'

# For testing only: set your API key here (do NOT hardcode in production)
os.environ["DASHSCOPE_API_KEY"] = "sk-545a143313614707ae22d68658e0d80a"

# Example data
today = datetime.today().strftime('%Y-%m-%d')
inventory = {
    "Product A": 25,
    "Product B": 10,
    "Product C": 0
}
lstm = {
    "Product A": 100,
    "Product B": 20,
    "Product C": 10
    
}


# Prepare biz_params with your custom data
biz_params = {
    "lstm": lstm, "inventory": inventory, "date":today
}

prompt_text = "I want analysis"

response = Application.call(
    api_key="sk-545a143313614707ae22d68658e0d80a",
    app_id='a58f8b59b1ae482e8368c7ac0b23a5c6',  # replace with your actual app id
    prompt=prompt_text,
    biz_params=biz_params
)

if response.status_code != HTTPStatus.OK:
    print(f'request_id={response.request_id}')
    print(f'code={response.status_code}')
    print(f'message={response.message}')
    print('Refer to: https://www.alibabacloud.com/help/en/model-studio/developer-reference/error-code')
else:
    print(response.output.text)


The key date to focus on is 5/5, which has already passed, and the next significant date is 6/6. However, the most important upcoming date is **11/11 (Singles’ Day)**, known for major sales and discounts in Malaysia and globally.

(##$FORCAST$##) 

Please choose from the four buttons for further in-depth analysis.


In [4]:
import requests

# URL of your FastAPI service (adjust if hosted elsewhere)
url = "http://localhost:3333/api/chatbot/"

# Sample payload
payload = {
    "prompt": "Give me inventory forecast analysis for today.",
    "inventory": {
        "Product A": 10,
        "Product B": 3,
        "Product C": 0
    },
    "lstm": {
        "Product A": 30,
        "Product B": 20,
        "Product C": 5
    },
    "date": "2025-05-17"
}

# Make the POST request
response = requests.post(url, json=payload)

# Print the result
if response.status_code == 200:
    print("✅ Response from chatbot:")
    print(response.json()["response"])
else:
    print("❌ Error:")
    print(response.status_code)
    print(response.text)

✅ Response from chatbot:
📦 Inventory Forecast Analysis for Today:

➤ **Product A**: 10 units in stock

➤ **Product B**: 3 units left — low stock!

➤ **Product C**: Out of stock with 0 units


In [ ]:
uvicorn server:app --host 0.0.0.0 --port 3333 --reload  